In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.execution import *
import pandas as pd
import time
import threading

In [2]:
# แก้ตรงนี้ก่อนนะครับ ให้ตรงตาม account ของแต่ละท่าน
ib_acct = "DUN463998" #update the ib account (different from real account and paper account) 
tickers = ["TSAT","TSLT","TSLR","CRWV"] #pick tickers with highest gap up or gap down
pos_size = 3000
profit_limit = 1000
loss_limit = -500


In [3]:
class TradeApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.pos_df = pd.DataFrame(columns=['Account', 'Symbol', 'SecType', 'Currency', 'Position', 'Avg cost'])
        self.order_df = pd.DataFrame(columns=['PermId', 'ClientId', 'OrderId',
                                  'Account', 'Symbol', 'SecType',
                                  'Exchange', 'Action', 'OrderType',
                                  'TotalQty', 'CashQty', 'LmtPrice',
                                  'AuxPrice', 'Status'])
        self.execution_df = pd.DataFrame(columns=['ReqId', 'PermId', 'Symbol',
                                          'SecType', 'Currency', 'ExecId',
                                          'Time', 'Account', 'Exchange',
                                          'Side', 'Shares', 'Price',
                                          'AvPrice', 'cumQty', 'OrderRef'])    
        self.hist_data = {}    
        self.last_price = {}
        self.hi_price = {}
        self.lo_price = {}
        self.pos_pnl = {}
        self.contract_id = {}
        self.av_volume = {}

##### wrapper function for reqMktData. this function handles streaming market data  (last current price)
    def tickPrice(self, reqId, tickType, price, attrib):    
        super().tickPrice(reqId, tickType, price, attrib)
        #print("TickPrice. TickerId:", reqId, "tickType:", tickType, "Price:", price)
        if tickType == 4:
            self.last_price[reqId] = price
                     
 ####  wrapper function for reqIds. this function manages the Order ID.
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)

#####   wrapper function for reqContractDetails. this function gives the contract ID for a given contract to be used in requesting PnL.
    def contractDetails(self, reqId, contractDetails):
        sym = contractDetails.contract.symbol
        con_id = contractDetails.contract.conId
        self.contract_id[sym] = con_id

#####   wrapper function for reqHistoricalData. this function gives the candle historical data
    def historicalData(self, reqId, bar):
        row = {"Date": bar.date, "Open": bar.open, "High": bar.high,
               "Low": bar.low, "Close": bar.close, "Volume": bar.volume}
        if reqId not in self.hist_data:
            # create once, then append with .loc
            self.hist_data[reqId] = pd.DataFrame(columns=["Date","Open","High","Low","Close","Volume"])
            self.hist_data[reqId].loc[0] = row
        else:
            self.hist_data[reqId].loc[len(self.hist_data[reqId])] = row
        print(f"reqID:{reqId}, date:{bar.date}, open:{bar.open}, high:{bar.high}, low:{bar.low}, close:{bar.close}, volume:{bar.volume}")

#####   wrapper function for reqHistoricalData. this function triggers when historical data extraction is completed      
    def historicalDataEnd(self, reqId, start, end):
        super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        ticker_event.set()
        if reqId == len(tickers) - 1:
            hist_event.clear()

    def contractDetailsEnd(self, reqId):
        super().contractDetailsEnd(reqId)
        print(f"contractDetailsEnd for reqId={reqId}")

#####   wrapper function for reqPositions.   this function gives the current positions
    def position(self, account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        dictionary = {"Account": account, "Symbol": contract.symbol, "SecType": contract.secType,
                      "Currency": contract.currency, "Position": position, "Avg cost": avgCost}
        if self.pos_df["Symbol"].str.contains(contract.symbol).any():
            self.pos_df.loc[self.pos_df["Symbol"] == contract.symbol, "Position"] = dictionary["Position"]
            self.pos_df.loc[self.pos_df["Symbol"] == contract.symbol, "Avg cost"] = dictionary["Avg cost"]
        else:
            self.pos_df.loc[len(self.pos_df)] = dictionary

#####   wrapper function for reqExecutions.   this function gives the executed orders                
    def execDetails(self, reqId, contract, execution):
        super().execDetails(reqId, contract, execution)
        dictionary = {"ReqId": reqId, "PermId": execution.permId, "Symbol": contract.symbol, "SecType": contract.secType,
                      "Currency": contract.currency, "ExecId": execution.execId, "Time": execution.time,
                      "Account": execution.acctNumber, "Exchange": execution.exchange, "Side": execution.side,
                      "Shares": execution.shares, "Price": execution.price, "AvPrice": execution.avgPrice,
                      "cumQty": execution.cumQty, "OrderRef": execution.orderRef}
        self.execution_df.loc[len(self.execution_df)] = dictionary
                
#####   this function is operated when the function reqPnLSingle is called. this function gives the p&L of each Ticker
    def pnlSingle(self, reqId, pos, dailyPnL, unrealizedPnL, realizedPnL, value):
        super().pnlSingle(reqId, pos, dailyPnL, unrealizedPnL, realizedPnL, value)
        self.pos_pnl[reqId] = dailyPnL           

#### wrapper function for reqOpenOrders. this function gives the open orders
    def openOrder(self, orderId, contract, order, orderState):
        super().openOrder(orderId, contract, order, orderState)
        dictionary = {"PermId": order.permId, "ClientId": order.clientId, "OrderId": orderId,
                      "Account": order.account, "Symbol": contract.symbol, "SecType": contract.secType,
                      "Exchange": contract.exchange, "Action": order.action, "OrderType": order.orderType,
                      "TotalQty": order.totalQuantity, "CashQty": order.cashQty,
                      "LmtPrice": order.lmtPrice, "AuxPrice": order.auxPrice, "Status": orderState.status}
        self.order_df.loc[len(self.order_df)] = dictionary

    def inExec(self,ticker):
        if len(self.execution_df[self.execution_df["Symbol"]==ticker]) == 0:
            return 0
        else:
            return -1 
        
    def tickerAllOpenOrders(self,ticker):
        return len(self.order_df[self.order_df["Symbol"]==ticker])
        
        

In [4]:
####  this function declares the properties of the instrument. 
def usStk(symbol,sec_type="STK",currency="USD",exchange="SMART"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.currency = currency
    contract.exchange = exchange
    return contract

####  this function declare market orders type. 
def marketOrder(direction,quantity):
    order = Order()
    order.action = direction
    order.orderType = "MKT"
    order.totalQuantity = quantity
    return order

####  this function declare BracketOrder orders type. 
def BracketOrder(parentOrderId, action, quantity, takeProfitLimitPrice, stopLossPrice):
    parent = Order()
    parent.orderId = parentOrderId
    parent.action = action
    parent.orderType = "MKT"
    parent.totalQuantity = quantity
    parent.transmit = False

    takeProfit = Order()
    takeProfit.orderId = parent.orderId + 1
    takeProfit.action = "SELL" if action == "BUY" else "BUY"
    takeProfit.orderType = "LMT"
    takeProfit.totalQuantity = quantity
    takeProfit.lmtPrice = takeProfitLimitPrice
    takeProfit.parentId = parentOrderId
    takeProfit.transmit = False

    stopLoss = Order()
    stopLoss.orderId = parent.orderId + 2
    stopLoss.action = "SELL" if action == "BUY" else "BUY"
    stopLoss.orderType = "STP"
    stopLoss.auxPrice = stopLossPrice
    stopLoss.totalQuantity = quantity
    stopLoss.parentId = parentOrderId
    stopLoss.transmit = True

    bracketOrder = [parent, takeProfit, stopLoss]
    return bracketOrder 

####  this function starts the streaming data of current ticker.
def streamSnapshotData(req_num,contract):
    """stream tick leve data"""
    app.reqMktData(reqId=req_num, 
                   contract=contract,
                   genericTickList="",
                   snapshot=False,
                   regulatorySnapshot=False,
                   mktDataOptions=[])

####  this function refreshes the order dataframe				   
def OrderRefresh(app):
    app.order_df = pd.DataFrame(columns=['PermId', 'ClientId', 'OrderId',
                                         'Account', 'Symbol', 'SecType',
                                         'Exchange', 'Action', 'OrderType',
                                         'TotalQty', 'CashQty', 'LmtPrice',
                                         'AuxPrice', 'Status'])
    app.reqOpenOrders()
    time.sleep(2)

def execRefresh(app):
    app.execution_df = pd.DataFrame(columns=['ReqId', 'PermId', 'Symbol',
                                             'SecType', 'Currency', 'ExecId',
                                             'Time', 'Account', 'Exchange',
                                             'Side', 'Shares', 'Price',
                                             'AvPrice', 'cumQty', 'OrderRef'])
    app.reqExecutions(21, ExecutionFilter())
    time.sleep(2)

    
def kill_switch(app):
        print("Kill Switch Activated!! Total day Pnl = {}".format(sum(app.pos_pnl.values())))
        app.reqGlobalCancel()
        app.reqIds(-1)
        time.sleep(2)
        order_id = app.nextValidOrderId
        pos_df = app.pos_df
        for ticker in pos_df["Symbol"]:
            quantity = pos_df[pos_df["Symbol"]==ticker]["Position"].values[0]
            if quantity > 0:
                app.placeOrder(order_id,usStk(ticker),marketOrder("SELL",quantity)) # EClient function to request contract details
            if quantity < 0:
                app.placeOrder(order_id,usStk(ticker),marketOrder("BUY",abs(quantity))) 
            order_id+=1
        print("Program Shutting Down!!")
        #exit()

def fetchHistorical(app):
    starttime = time.time()
    first_pass = True
    while not kill_event.is_set():
        hist_event.set()
        app.hist_data = {}
        for ticker in tickers:
            ticker_event.clear()
            app.reqHistoricalData(reqId=tickers.index(ticker), 
                                  contract=usStk(ticker),
                                  endDateTime='',
                                  durationStr="5 D" if first_pass else "1 D",
                                  barSizeSetting="15 mins" if first_pass else "5 mins",
                                  whatToShow='ADJUSTED_LAST',
                                  useRTH=1,
                                  formatDate=1,
                                  keepUpToDate=0,
                                  chartOptions=[])
            ticker_event.wait()
            if first_pass:
                tot_vol = app.hist_data[tickers.index(ticker)]["Volume"].astype(int).sum()
                num = len(app.hist_data[tickers.index(ticker)]["Volume"])
                app.av_volume[ticker] = int(tot_vol/(num*3))
                app.hi_price[ticker] = app.hist_data[tickers.index(ticker)].iloc[-1]["High"]
                app.lo_price[ticker] = app.hist_data[tickers.index(ticker)].iloc[-1]["Low"]
        first_pass = False
        time.sleep(300 - ((time.time() - starttime) % 300.0))    

        
def openRangeBrkout(app):
    while not kill_event.is_set():
        for ticker in tickers:
            OrderRefresh(app)
            execRefresh(app)
            current_tot_pnl = sum(app.pos_pnl.values())
            [hour , minute] = time.strftime("%H %M").split() #getting local system time
            print ("local time - hour {} minute {} ".format(hour,minute))
            
            if current_tot_pnl > profit_limit or current_tot_pnl < loss_limit or ((int(hour) >= 22 and int(minute) > 30)):
                kill_event.set()
                kill_switch(app)
                continue
         
            if app.inExec(ticker) == 0 and app.tickerAllOpenOrders(ticker) == 0 and not hist_event.is_set():
                last_volume = app.hist_data[tickers.index(ticker)].iloc[-1]["Volume"]
                if 2*app.av_volume[ticker] < last_volume:
                    if app.last_price[tickers.index(ticker)] > app.hi_price[ticker]:
                        quantity = int(pos_size/app.last_price[tickers.index(ticker)])
                        tp_price = round(app.last_price[tickers.index(ticker)]*1.05,2)
                        sl_price = app.lo_price[ticker]
                        
                        app.reqIds(-1)
                        time.sleep(2)
                        order_id = app.nextValidOrderId
                        bracket = BracketOrder(order_id,"BUY",quantity,tp_price,sl_price)
                        for o in bracket:
                            app.placeOrder(o.orderId, usStk(ticker), o)
                            
                            
                    if app.last_price[tickers.index(ticker)] < app.lo_price[ticker]:
                        quantity = int(pos_size/app.last_price[tickers.index(ticker)])
                        tp_price = round(app.last_price[tickers.index(ticker)]*0.95,2)
                        sl_price = app.hi_price[ticker]
                        
                        app.reqIds(-1)
                        time.sleep(2)
                        order_id = app.nextValidOrderId
                        bracket = BracketOrder(order_id,"SELL",quantity,tp_price,sl_price)
                        for o in bracket:
                            app.placeOrder(o.orderId, usStk(ticker), o)
                            
        time.sleep(15)
    
##### function to establish the websocket connection to TWS
def connection():
    app.run()

In [ ]:
app = TradeApp()
app.connect(host='127.0.0.1', port=7497, clientId=22) #port 4002 for ib gateway paper trading/7497 for TWS paper trading

ConThread = threading.Thread(target=connection)
ConThread.start()

kill_event = threading.Event()
ticker_event = threading.Event()
hist_event = threading.Event()

for ticker in tickers:
    # Ask for contract details
    app.reqContractDetails(tickers.index(ticker), usStk(ticker))
    # Wait up to ~5s for the callback to fill app.contract_id[ticker]
    for _ in range(50):
        if ticker in app.contract_id:
            break
        time.sleep(0.1)
    if ticker not in app.contract_id:
        print(f"[WARN] Contract ID not ready for {ticker}; skipping PnL subscription for now.")
        continue

    # Now it’s safe to proceed
    time.sleep(0.2)
    streamSnapshotData(tickers.index(ticker), usStk(ticker))
    time.sleep(0.2)
    app.reqPnLSingle(tickers.index(ticker), ib_acct, "", app.contract_id[ticker])
    time.sleep(0.2)
    
histdataTread = threading.Thread(target = fetchHistorical, args=(app,))
histdataTread.start()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


NextValidId: 1
contractDetailsEnd for reqId=0


ERROR 0 354 Requested market data is not subscribed. Check API status by selecting the Account menu then under Management choose Market Data Subscription Manager and/or availability of delayed data.Delayed market data is available.TSAT NASDAQ.NMS/TOP/ALL


contractDetailsEnd for reqId=1


ERROR 1 354 Requested market data is not subscribed. Check API status by selecting the Account menu then under Management choose Market Data Subscription Manager and/or availability of delayed data.Delayed market data is available.TSLT BATS/TOP/ALL


...

reqID:1, date:20250910  20:30:00, open:18.28, high:18.43, low:17.82, close:17.9, volume:4043
reqID:1, date:20250910  20:45:00, open:17.89, high:18.12, low:17.83, close:17.98, volume:3304
reqID:1, date:20250910  21:00:00, open:17.98, high:18.36, low:17.97, close:18.14, volume:2418
reqID:1, date:20250910  21:15:00, open:18.14, high:18.27, low:18.1, close:18.13, volume:1798
reqID:1, date:20250910  21:30:00, open:18.12, high:18.41, low:18.12, close:18.41, volume:1978
reqID:1, date:20250910  21:45:00, open:18.43, high:18.78, low:18.37, close:18.78, volume:2467
reqID:1, date:20250910  22:00:00, open:18.77, high:18.83, low:18.63, close:18.83, volume:2209
reqID:1, date:20250910  22:15:00, open:18.83, high:18.86, low:18.63, close:18.71, volume:1673
reqID:1, date:20250910  22:30:00, open:18.7, high:18.74, low:18.58, close:18.73, volume:781
reqID:1, date:20250910  22:45:00, open:18.72, high:18.81, low:18.69, close:18.72, volume:699
reqID:1, date:20250910  23:00:00, open:18.71, high:18.73, low:18.

ERROR 2 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 1521.4 to 1521


reqID:2, date:20250910  20:30:00, open:21.6, high:21.81, low:21.09, close:21.2, volume:1521
reqID:2, date:20250910  20:45:00, open:21.19, high:21.45, low:21.11, close:21.28, volume:732
reqID:2, date:20250910  21:00:00, open:21.28, high:21.73, low:21.27, close:21.46, volume:660
reqID:2, date:20250910  21:15:00, open:21.49, high:21.63, low:21.43, close:21.44, volume:284
reqID:2, date:20250910  21:30:00, open:21.46, high:21.81, low:21.46, close:21.8, volume:961
reqID:2, date:20250910  21:45:00, open:21.83, high:22.23, low:21.77, close:22.23, volume:1514
reqID:2, date:20250910  22:00:00, open:22.22, high:22.3, low:22.07, close:22.3, volume:1051
reqID:2, date:20250910  22:15:00, open:22.3, high:22.33, low:22.07, close:22.14, volume:749
reqID:2, date:20250910  22:30:00, open:22.14, high:22.18, low:22.0, close:22.18, volume:591
reqID:2, date:20250910  22:45:00, open:22.19, high:22.27, low:22.12, close:22.18, volume:405
reqID:2, date:20250910  23:00:00, open:22.13, high:22.17, low:22.05, close

ERROR 3 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 101691.01 to 101691


reqID:3, date:20250910  20:30:00, open:110.03, high:118.75, low:109.01, close:117.84, volume:101691
reqID:3, date:20250910  20:45:00, open:117.77, high:124.42, low:117.58, close:124.16, volume:88861
reqID:3, date:20250910  21:00:00, open:124.16, high:124.9, low:118.35, close:119.37, volume:45722
reqID:3, date:20250910  21:15:00, open:119.38, high:120.35, low:117.28, close:119.89, volume:30941
reqID:3, date:20250910  21:30:00, open:119.96, high:121.9, low:119.1, close:119.24, volume:22444
reqID:3, date:20250910  21:45:00, open:119.3, high:120.88, low:118.52, close:118.55, volume:13343
reqID:3, date:20250910  22:00:00, open:118.57, high:120.03, low:118.33, close:119.65, volume:12299
reqID:3, date:20250910  22:15:00, open:119.64, high:120.67, low:119.2, close:119.42, volume:13980
reqID:3, date:20250910  22:30:00, open:119.47, high:119.67, low:118.2, close:118.82, volume:10903
reqID:3, date:20250910  22:45:00, open:118.77, high:119.63, low:118.22, close:119.28, volume:9412
reqID:3, date:20

In [6]:
startegyThread = threading.Thread(target = openRangeBrkout, args =(app,))
startegyThread.start()

local time - hour 21 minute 00 
local time - hour 21 minute 00 
local time - hour 21 minute 00 


Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\TunKedsaro\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\TunKedsaro\AppData\Roaming\Python\Python39\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\TunKedsaro\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\TunKedsaro\AppData\Local\Temp\ipykernel_18020\2959486428.py", line 141, in openRangeBrkout
KeyError: 3


local time - hour 21 minute 00 
